In [ ]:
%%capture
%run 02_FeatEng.ipynb

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error  as skl_mse

In [ ]:
import mle.tsa as mle_tsa

In [ ]:
import shap 
shap.initjs()

# Split Data

In [ ]:
all_features = curr_features + seas_features + extra_features
target = 'cons_GNA95_pct_change1_t1'

In [ ]:
x_train = data_train[all_features].copy()
y_train = data_train[target].copy()

In [ ]:
x_test = data_test[all_features].copy()
y_test = data_test[target].copy()

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)

In [ ]:
fig, axs = plt.subplots(3,1, sharex=True)

for i, (train_index, val_index) in enumerate(tscv.split(y_train)):
    axs[i].plot(y_train[train_index],label='train')
    axs[i].plot(y_train[val_index],label='valid')

# Model Building

In [ ]:
rf  = RandomForestRegressor(criterion='mse', n_estimators=100, bootstrap=True, oob_score=True, ccp_alpha=0,
                            min_samples_leaf=3,
                            n_jobs=-1, random_state=123)

In [ ]:
rf_hparams_grid = {'max_depth': [2, 4],
                    'max_features': [0.5, 0.75]}
rf_cv = GridSearchCV(rf, rf_hparams_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=tscv, refit=True, return_train_score=True)
rf_cv_res = rf_cv.fit(x_train, y_train)

In [ ]:
rf_cv_res_df = pd.DataFrame(rf_cv_res.cv_results_)

In [ ]:
rf_cv_mod = rf_cv_res.best_estimator_

In [ ]:
rf_cv_mod

In [ ]:
rf_cv_res.best_score_

## Prediction and Forecating

In [ ]:
p_gna95_train = pd.Series(index=data_train.index, data=rf_cv_mod.predict(x_train))
p_gna95_test = pd.Series(index=data_test.index, data=rf_cv_mod.predict(x_test))

In [ ]:
ax = p_gna95_test.plot()
y_test.plot(label='actual', style='o', ax=ax)
plt.show()

In [ ]:
p_gna95_train_df = p_gna95_train.to_frame("p_cons_GNA95_pct_change1_t1")
p_gna95_train_df['cons_GNA95'] = data_train['cons_GNA95']

p_gna95_test_df = p_gna95_test.to_frame("p_cons_GNA95_pct_change1_t1")
p_gna95_test_df['cons_GNA95'] = data_test['cons_GNA95']

p_gna95_train_df['split'] = 'train'
p_gna95_test_df['split'] = 'test'

p_gna95_df = pd.concat([p_gna95_train_df, p_gna95_test_df], axis=0)

In [ ]:
p1_s = pd.Series(index=p_gna95_df.index, dtype=np.number)
p0 = p_gna95_df['cons_GNA95'].iloc[0]

for idx, row in p_gna95_df.iterrows():
    p1 = (row['p_cons_GNA95_pct_change1_t1'] * p0 + p0)
    p0 =  p_gna95_df.loc[idx,'cons_GNA95']  # Using true value, carefully verify this assumption in your workflow
    p1_s[idx] = p1


In [ ]:
p_gna95_df['cons_GNA95_t1'] = p_gna95_df['cons_GNA95'].shift(-1)
p_gna95_df["p_cons_GNA95_t1"] = p1_s
# Adatp to forecasting models time-frame
p_gna95_df["p_cons_GNA95"] = p_gna95_df["p_cons_GNA95_t1"].shift(1)
p_gna95_df = p_gna95_df.iloc[1:].copy()

In [ ]:
fig, ax = plt.subplots(1,1)
p_gna95_df.plot(y = 'p_cons_GNA95_t1', label='forecast', ax=ax)
p_gna95_df.plot(y = 'cons_GNA95_t1', label='actual', style='o', ax=ax)
plt.show()

# Model Interpretation

## Global feature importance
* Measures how important a feature is in overal model performance
* scikit-learn out-of-the box global feature imoprtance does not inform about relationship direction

In [ ]:
sklearn_fi = pd.Series(index=all_features, data=rf_cv_mod.feature_importances_).sort_values(ascending=True)
sklearn_fi.plot.barh(figsize =(8,6))
plt.title('sklearn feature importance')
plt.show()

## SHAP values

In [ ]:
explainer_model1 = shap.TreeExplainer(rf_cv_mod, feature_perturbation ='tree_path_dependent')
shap_values1 = explainer_model1.shap_values(x_train)

In [ ]:
explainer_model2 = shap.TreeExplainer(rf_cv_mod, data =x_train)
shap_values2 = explainer_model2.shap_values(x_train)

In [ ]:
print('Expected base value in shap algorithm: ', explainer_model1.expected_value)
print('Expected base value in shap algo, fit with data: ', explainer_model2.expected_value)
print('Target mean: ', y_train.mean())
print('Model prediction mean: ', p_gna95_train.mean())

In [ ]:
shap.summary_plot(shap_values2, x_train, plot_type="bar")

In [ ]:
pd.Series(index=all_features, data=abs(shap_values2).mean(axis=0)).sort_values().plot.barh(figsize=(8,6))
plt.title("GNA95 Random forest feature importance")
plt.show()

In [ ]:
pd.Series(index=all_features, data=shap_values2.mean(axis=0)).sort_values().plot.barh()
plt.show()

## Local Feature Importance

In [ ]:
shap.force_plot(explainer_model2.expected_value, shap_values2[3], x_train.iloc[[3]], figsize =(16,4))

In [ ]:
def shap_force_plot(base_value,  shap_values, features_data, obs_j):
    features_names = features_data.columns.tolist()
    shap_values_row = shap_values[obs_j]
    features_row = features_data.iloc[obs_j]
    shap_vals_j = pd.Series(index=features_names, data=shap_values_row).to_frame('raw shap values j')
    shap_vals_j['shap values j'] = shap_vals_j['raw shap values j'] + base_value
    shap_vals_j['contrib to base value'] = np.where(shap_vals_j['raw shap values j']>0, 'positive', 'negative')
    ax = sns.barplot(data=shap_vals_j, x='raw shap values j', y=features_names, hue='contrib to base value')
    plt.title('Force plot: Shap contribution to prediction by feature')
    return ax

In [ ]:
shap_values_df = pd.DataFrame(index=x_train.index, data=shap_values2, columns=all_features)
shap_values_df

## Detailed Analysis: z-std pct changes in consumption influence
z_std measures a shudden and important change in consumption, in general, it is less important than other features like seasonality, but can be a main driver forecast in some extreme situtations?

In [ ]:
sns.distplot(shap_values_df['cons_GNA95_pct_change1_zstd'])
plt.title("Distribution of z-std pct changes in consumption influence on prediction")
plt.show()

In [ ]:
most_imp_feat_by_obs = abs(shap_values_df).idxmax(axis=1)

In [ ]:
mask = most_imp_feat_by_obs[most_imp_feat_by_obs == 'cons_GNA95_pct_change1_zstd'] 
p_gna95_df.loc[mask.index]

In [ ]:
idx = range(len(most_imp_feat_by_obs))
dates = most_imp_feat_by_obs.index.strftime('%Y-%m-%d')
dt_to_idx = dict(zip(dates, idx))

In [ ]:
date_obs = '2013-07-01'
idx_obs = dt_to_idx[date_obs]
shap.force_plot(explainer_model2.expected_value, shap_values2[idx_obs], x_train.iloc[[idx_obs]], figsize =(16,4))

In [ ]:
x_feat_value = x_train.loc[date_obs, 'cons_GNA95_pct_change1_zstd']
sns.distplot(x_train['cons_GNA95_pct_change1_zstd'])
plt.title("Distribution of z-std pct changes in consumption feature value")
plt.axvline(x_feat_value)
plt.text(0.80, 0.95, f'date: {date_obs}', ha='right', va='top', transform = ax.transAxes)
plt.show()

In [ ]:
x_train

In targeted, pct change in consumptions experiments a tail value (in z_std scale), this leads the model to a prediction where this feature is the MOST IMPORTANT CONTRIBUTOR. In addition, this contribution leads the prediction really close to true value

In [ ]:
zoom_yr = int(date_obs[:4])

fig, axs = plt.subplots(2,1)
plt.suptitle(f"Targeted: {date_obs} prediction: pct change in GNA95 consumption")

ax = axs[0]
p_gna95_train_df['2012':'2014'].plot(y='p_cons_GNA95_pct_change1_t1',ax=ax)
y_train['2012':'2014'].plot(y='cons_GNA95_pct_change1_t1', style='o', ax=ax)
ax = axs[1]
x_train['2012':'2014'].plot(y='cons_GNA95_pct_change1_zstd', ax=ax)

for ax in axs: 
    ax.axvline(date_obs, color='orange')
    

plt.show()

In [ ]:
ax = shap_force_plot(base_value=explainer_model2.expected_value,  shap_values=shap_values2, features_data=x_train, obs_j=idx_obs)
plt.show()

In [ ]:
x_train.iloc[idx_obs:idx_obs+1].T

# Benchmark

In [ ]:
mask_test = p_gna95_df['split'] == 'test'
skl_mse(p_gna95_df.loc[mask_test, 'cons_GNA95'], p_gna95_df.loc[mask_test, 'p_cons_GNA95'])**0.5

In [ ]:
mask_test = p_gna95_df['split'] == 'test'
mle_tsa.compute_ape(p_gna95_df.loc[mask_test, 'cons_GNA95'], p_gna95_df.loc[mask_test, 'p_cons_GNA95']).mean()

In [ ]:
select_cols = ['model', 'split', 'cons_GNA95', 'p_cons_GNA95']

p_gna95_df['model'] = 'simple_random_forest'
p_gna95_df[select_cols].to_excel(OUTPATH + OUTFILE, index_label='Date')